In [43]:
# Given a list of audit entries. Determine field changes from one row to the next
import pandas as pd

In [44]:
d2 = {'transaction_id': [1,1, 2],
    'account_number': ['12345', '12345', '99999'],
    'amount': [10.5, 9, 100],
    'last_action': ['I', 'U', 'I'],
    'last_modified_timestamp' : ['2023-01-01 00:00:00', '2023-01-01 12:00:00', '2023-01-01 00:00:00']
    }
df1 = pd.DataFrame(data=d2)
df1

,transaction_id,account_number,amount,last_action,last_modified_timestamp
0,1,12345,10.5,I,2023-01-01 00:00:00
1,1,12345,9.0,U,2023-01-01 12:00:00
2,2,99999,100.0,I,2023-01-01 00:00:00


In [45]:
df1 = pd.read_csv('transaction_audit_in.csv')
df1

,transaction_id,account_number,amount,last_action,last_modified_timestamp
0,1,12345,10.0,I,2023-01-01 00:00:00
1,1,12345,20.5,U,2023-01-01 12:00:00
2,1,12345,20.2,U,2023-01-01 12:30:00
3,2,99999,23.8,I,2023-01-01 00:00:00


In [46]:
df_transaction = df1[df1['transaction_id'] == 1].sort_values(by=['last_modified_timestamp'], ascending=True)
df_transaction = df_transaction.melt(id_vars=['transaction_id', 'last_action', 'last_modified_timestamp'], 
    value_vars=['amount', 'account_number'], 
    var_name='field_name', value_name='field_value')
df_transaction.sort_values(by=['field_name', 'last_modified_timestamp'], ascending=True)

,transaction_id,last_action,last_modified_timestamp,field_name,field_value
3,1,I,2023-01-01 00:00:00,account_number,12345.0
4,1,U,2023-01-01 12:00:00,account_number,12345.0
5,1,U,2023-01-01 12:30:00,account_number,12345.0
0,1,I,2023-01-01 00:00:00,amount,10.0
1,1,U,2023-01-01 12:00:00,amount,20.5
2,1,U,2023-01-01 12:30:00,amount,20.2


In [54]:
df_transaction["field_value_previous"] = df_transaction.groupby(['field_name'])["field_value"].shift(1).fillna('N/A')
df_transaction

,transaction_id,last_action,last_modified_timestamp,field_name,field_value,field_value_previous
0,1,I,2023-01-01 00:00:00,amount,10.0,N/A
1,1,U,2023-01-01 12:00:00,amount,20.5,10.0
2,1,U,2023-01-01 12:30:00,amount,20.2,20.5
3,1,I,2023-01-01 00:00:00,account_number,12345.0,N/A
4,1,U,2023-01-01 12:00:00,account_number,12345.0,12345.0
5,1,U,2023-01-01 12:30:00,account_number,12345.0,12345.0


In [48]:
df_transaction.to_csv('transaction_audit_out.csv', index=False)

In [55]:
df_updates = df_transaction[(df_transaction['field_value_previous'] != 'N/A') & (df_transaction['field_value'] != df_transaction['field_value_previous'])]
df_updates

,transaction_id,last_action,last_modified_timestamp,field_name,field_value,field_value_previous
1,1,U,2023-01-01 12:00:00,amount,20.5,10.0
2,1,U,2023-01-01 12:30:00,amount,20.2,20.5
